In [74]:
import MeCab
from transformers import ElectraModel, ElectraTokenizer
import torch
import pandas as pd
from tqdm import tqdm
from transformers import ElectraForTokenClassification

# 1. 데이터 읽어오기

In [ ]:
claim_data_path = "./data/patent/kipris_claim_patent.csv"
claim_df = pd.read_csv(claim_data_path)
claim_df = claim_df.drop(columns=["Unnamed: 0"])
claim_df.head()

# 2. 개체명 추출

## 2.1 mecab-ko-dic

In [64]:
# path = "/home/ui/project/graduate_paper/korpatElectra/KIPI-KorPatELECTRA/pat_all_mecab_dic.csv"
# user_dict = pd.read_csv(path)
# len(user_dict), user_dict.head()

(6663692,
     실시예  Unnamed: 1  Unnamed: 2  Unnamed: 3  NNG  *  F 실시예.1 *.1 *.2 *.3 *.4
 0    일측         NaN         NaN         NaN  NNG  *  T    일측   *   *   *   *
 1   청구항         NaN         NaN         NaN  NNG  *  T   청구항   *   *   *   *
 2   제어부         NaN         NaN         NaN  NNG  *  F   제어부   *   *   *   *
 3  어드레스         NaN         NaN         NaN  NNG  *  F  어드레스   *   *   *   *
 4  제조방법         NaN         NaN         NaN  NNG  *  T  제조방법   *   *   *   *)

In [8]:
# Tagger 객체 생성
mecab = MeCab.Tagger()
print(mecab.parse("안녕하세요."))

안녕	NNG,행위,T,안녕,*,*,*,*
하	XSV,*,F,하,*,*,*,*
세요	EP+EF,*,F,세요,Inflect,EP,EF,시/EP/*+어요/EF/*
.	SF,*,*,*,*,*,*,*
EOS



In [ ]:
columns = ["품사태그", "의미 부류", "종성유무", "원형", "타입", "첫번째품사", "마지막품사", "표현"]
result_df = pd.DataFrame()

for data in claim_df['claim']:
    for d in mecab.parse(data).split("\n"):
        try:
            text, classification = d.split("\t")
            result_list = classification.split(",")
            result_df = pd.concat([result_df, pd.DataFrame([result_list], columns=columns)])
            
        except ValueError:  # 마지막에 공백이 split("\t") 안 됨
            continue
        except Exception as e:
            print(e)

    break

In [61]:
result_df.head()

,품사태그,의미 부류,종성유무,원형,타입,첫번째품사,마지막품사,표현
0,SN,*,*,*,*,*,*,*
0,SF,*,*,*,*,*,*,*
0,NR,*,F,하나,*,*,*,*
0,NNG,*,T,이상,*,*,*,*
0,NNG,*,T,배전반,Compound,*,*,배전/NNG/*+반/NNG/*


In [ ]:
# result_df.to_csv("./data/output/electra_macab_test.csv")

## 2.2 BIO 태깅

# 2.3 모델 

In [76]:
# 모델과 토크나이저 로드
model_name = "monologg/koelectra-base-v3-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraModel.from_pretrained(model_name)
model = ElectraForTokenClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator",
    num_labels=10
)

# 입력 텍스트
text = "안녕하세요, Transformers 라이브러리를 사용하고 있습니다."

# 토큰화 (딕셔너리 형태로 출력)
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# 모델 출력
with torch.no_grad():
    outputs = model(**inputs)

# print(outputs.last_hidden_state.shape)

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for data in claim_df['claim']: # 청구항 4줄 이상
    for sentence in data.split("\n"): # 청구항 1줄
            
        
        break
    break

1. 하나 이상 배전반, 조기화재검지센서, 감시카메라로 구성된 배전반 통합 방재시스템에 있어서,배전반은 통합제어계전기, 배전반 우선순위절체기, 단말 6G SECURITY ENGINE을 포함하는 저압반, 아크검출기를 포함하는 고압반, 교류모터 고장 검출장치를 포함하는 모터콘트롤반, 전압이상 차단기를 포함하는 분전반을 포함하는 배전반이며;상기 배전반을 감시하는 감시카메라는 주제어부, 영상 보안처리부, 입출력부를 포함하여 구성되어 통합제어계전기에서 턴온 이벤트, 동작 제어신호 이벤트, 이상신호 이벤트, 조기화재검지 이벤트가 수신될 경우, 주제어부는 MCU(Micro Control Unit), PUF Chip, PUF-PIN Memory, QRNG Chip, Security OTP Memory로 구성되며;영상 보안처리부는 촬상센서부, 5G QUANTUM JUMP ENGINE로 구성되며; 양자통신부는 Bob 6G QUANTUM SERVER, Non-QUANTUM CHANNEL, Alice 6G QUANTUM SERVER로 구성되어,촬상센서부 내부 이미지센서는 광학신호를 전기신호로 변환한 영상데이터를 5G QUANTUM JUMP ENGINE으로 전송하며;하나 이상의 배전반으로부터 수집되는 배전반 상태정보 데이터를 단말 6G SECURITY ENGINE에서 수집하여 Bob 6G QUANTUM SERVER로 전송하며;및Alice 6G QUANTUM SERVER는 배전반 우선순위절체기 제어데이터를 Bob 6G QUANTUM SERVER로 전송하여 배전반 내부 단말 6G SECURITY ENGINE을 통하여 제어하며;단말 6G SECURITY ENGINE는 Bob 6G QUANTUM SERVER로부터 배전반 우선순위절체기 제어데이터를 수신하여 배전반 우선순위절체기로 전송하며;및배전반 우선순위절체기 제어데이터에 따라 배전반 우선순위절체기를 ON/OFF 제어하여 하나 이상의 배전반을 선택 운전하며;및 주제어부 MCU(Micro Control Unit)는 PUF Chip의 하드웨어 